In [60]:
#1. Importo librerías.
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import time
import random

In [61]:
#2. Configuro constantes a definir para realizar el script.
pagina_inicio = 78 # 148 está la primera noticia de Enero.
pagina_final = 76 # 76 está la última noticia de Abril.

In [62]:
#3. Funciones de interés.
#a.
def get_news_list(page_url):
    response = requests.get(page_url) # Request la URL.
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser') # Scrappeo el contenido.
    noticias = soup.find_all('article') # Me quedo con las noticias.
    
    news_data = []
    for noticia in noticias: # Recorro cada noticia.
        title_tag = noticia.find('a')
        if title_tag:
            title = title_tag.get_text(strip=True)
            link = title_tag['href']
            if not link.startswith('http'):
                link = 'https://www.clarin.com' + link
            
            news_data.append({'title': title, 'link': link})
    return news_data

#b.
def get_news_content_and_date(news_url):
    response = requests.get(news_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraer fecha desde <time class="createDate" datetime="...">
    fecha = None
    time_tag = soup.find('time', class_='createDate')
    if time_tag and time_tag.has_attr('datetime'):
        fecha = time_tag['datetime']
    else:
        # Backup: buscar otras formas por si cambia el HTML
        time_tag = soup.find('time')
        if time_tag and time_tag.has_attr('datetime'):
            fecha = time_tag['datetime']
        elif time_tag:
            fecha = time_tag.get_text(strip=True)

    # Extraer título desde <h1 class="storyTitle">
    titulo_tag = soup.find('h1', class_='storyTitle')
    titulo = titulo_tag.get_text(strip=True) if titulo_tag else 'Título no encontrado'

    # Extraer contenido
    content_div = soup.find('div', {'class': ['body', 'article-body']})
    if not content_div:
        content_div = soup.find('article')

    if content_div:
        paragraphs = content_div.find_all('p')
        contenido = '\n'.join(p.get_text(strip=True) for p in paragraphs)
    else:
        contenido = "Contenido no encontrado"
    
    return fecha, titulo, contenido

#c.
def scrape_pages(start_page=pagina_inicio, end_page=pagina_final):
    diario = 'Clarín'
    output_dir = './outputs'
    os.makedirs(output_dir, exist_ok=True)
    
    for page_num in range(start_page, end_page - 1, -1):
        url = f'https://www.clarin.com/economia/page/{page_num}'
        print(f'Procesando página {page_num} ...')
        records = []
        
        try:
            news_list = get_news_list(url)
        except Exception as e:
            print(f"❌ Error al obtener lista de noticias en página {page_num}: {e}")
            continue
        
        for news in news_list:
            try:
                link = news['link']
                fecha, titulo_real, contenido = get_news_content_and_date(link)
                
                # Formatear fecha
                if fecha:
                    try:
                        fecha_dt = datetime.fromisoformat(fecha.replace('Z', '+00:00'))
                        fecha_str = fecha_dt.strftime('%Y-%m-%d')
                    except:
                        fecha_str = fecha
                else:
                    fecha_str = 'Fecha no encontrada'

                records.append({
                    'diario': diario,
                    'fecha': fecha_str,
                    'titulo noticia': titulo_real,
                    'contenido': contenido,
                    'link': link
                })
            except Exception as e:
                print(f"⚠️  Error al procesar noticia {news.get('link', '')}: {e}")
            
            tiempo_espera_noticias = random.randint(1,4)
            time.sleep(tiempo_espera_noticias)  # Espera entre noticias
        
        # Guardar CSV
        df = pd.DataFrame(records)
        output_path = os.path.join(output_dir, f'clarin_pag_{page_num}.csv')
        df.to_csv(output_path, index=False)
        print(f'✅ Página {page_num} guardada en {output_path}')
        
        tiempo_espera_paginas = random.randint(1,10)
        time.sleep(tiempo_espera_paginas)  # Espera entre páginas.

In [63]:
#4. Ejecuto la función para scrappear la página de Clarín.
if __name__ == "__main__":
    scrape_pages()
    
print("Terminó.")

Procesando página 78 ...
✅ Página 78 guardada en ./outputs\clarin_pag_78.csv
Procesando página 77 ...
✅ Página 77 guardada en ./outputs\clarin_pag_77.csv
Procesando página 76 ...
✅ Página 76 guardada en ./outputs\clarin_pag_76.csv
Terminó.
